# Choix du pas de temps

## Pas fixe

On commence par coder une fonction qui résout l'équation différentielle $\dot{x} = f(x)$ avec le schéma d'Euler explicite.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def solve_euler_explicit(f, x0, dt, t0, tf):
  n = int((tf-t0)//dt)
  t = np.linspace(t0, tf, n)
  x = np.zeros(n)
  x[0] = x0
  for j in range(1, n):
    x[j] = x[j-1] + dt*f(x[j-1])
  return t, x

### Exemple avec $f : x \mapsto x$

In [ ]:
def exp_solver(solver, dt, t0, tf = 5, x0 = 1):
    for delta_t in dt :
        t, x = solver(lambda x : x, x0, delta_t, t0, tf)
        plt.plot(t, x, label = f'dt = {delta_t}')
    plt.plot(t, [np.exp(i) for i in t], label = 'exp')
    plt.legend()
    plt.show()

In [ ]:
exp_solver(solve_euler_explicit, [0.1, 0.01], 0, 5)

In [ ]:
for dt in [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001] :
    t, x = solve_euler_explicit(lambda x : x, 1, dt, 0, 5)
    x_exp = [np.exp(i) for i in t]
    dif = np.abs(x-x_exp)
    print(f'Pour dt = {dt}, le max de la différence divisé par dt est d\'environ {round(np.max(dif)/dt)}.')
print('On en déduit que le schéma est bien convergeant à l\'ordre 1 avec c_v = 400.')


On choisit maintenant un schéma à l'ordre 2 : la méthode de Heun.

In [ ]:
def heun(f, x0, dt, t0, tf):
    n = int((tf-t0)/dt)
    t = np.linspace(0, dt*(n-1), n)
    x = np.zeros(n)
    x[0] = x0
    for j in range(1, n):
        x[j] = x[j-1] + dt/2*( f(x[j-1]) + f(x[j-1] + dt*f(x[j-1])) )
    return t, x

In [ ]:
exp_solver(heun, [0.1, 0.01], 0, 5)

Ce schéma est plus précis que celui d'Euler explicite. 

In [ ]:
for dt in [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001] :
    t, x = heun(lambda x : x, 1, dt, 0, 5)
    x_exp = [np.exp(i) for i in t]
    dif = np.abs(x-x_exp)
    print(f'Pour dt = {dt}, le max de la différence divisé par dt est d\'environ {round(np.max(dif)/(dt**2))}.')
print('On en déduit que le schéma de Heun est convergeant à l\'ordre 2 avec c_v = 125.')

## Adaptation du pas de temps

On définit $\tilde{x}$ comme étant la vraie solution initialisée par $x^j$ au temps $t_j$. On a ainsi : $ \forall t \in [ t_0, t_f], \tilde{x}(t) = x^j + \int_{t_j}^{t} f(s,\tilde{x}(s))ds$.

On a : 

$e^{j+1} = x^j + \int_{t_j}^{t_{j+1}} f(s,\tilde{x}(s))ds - x^{j+1} = x^j + \int_{t_j}^{t_{j+1}} f(s,\tilde{x}(s))ds - ( x^{j} + (t_{j+1} - t_{j})*f(t_j,x^j))$

Soit : $e^{j+1} = \int_{t_j}^{t_{j+1}} (f(s,\tilde{x}(s)) - f(t_j,x^j))ds$

Or si $f$ est $C^1$, on a, en faisant un développement à l'ordre 1 de $s \mapsto f(s,\tilde{x}(s))$ qui est elle-même $C^1$: $ \forall s \in [ t_j , t_{j+1} ],  f(s,\tilde{x}(s)) = f(t_j,\tilde{x}(t_j)) + [\frac{\partial f}{\partial x}(t_j,\tilde{x}(t_j))+ \tilde{x'}(t_j)\frac{\partial f}{\partial y}(t_j,\tilde{x}(t_j))]  \times (s - t_j) + O((s- t_j)^2) $

Or $\tilde{x}'(t_j) = f(t_j,\tilde{x}(t_j)) = f(t_j, x^j)$ donc, on a : $\forall s \in [ t_j , t_{j+1} ],  f(s,\tilde{x}(s)) = f(t_j,x^j) + [\frac{\partial f}{\partial x}(t_j,x^j)+ f(t_j,x^j)\frac{\partial f}{\partial y}(t_j,x^j)]  \times (s - t_j) + O((s- t_j)^2)$ 

Puis : $e^{j+1} = \int_{t_j}^{t_{j+1}} [\frac{\partial f}{\partial x}(t_j,x^j)+ f(t_j,x^j)\frac{\partial f}{\partial y}(t_j,x^j)]   \times (s - t_j) + O((s- t_j)^2) ds = [\frac{\partial f}{\partial x}(t_j,x^j)+ f(t_j,x^j)\frac{\partial f}{\partial y}(t_j,x^j)]\times \frac{\Delta t_j^2}{2} + O(\Delta t_j^3)$

Par ailleurs, on peut faire un faire un développement à l'ordre 1 de f en $(t_j,x(t_j))$ car f est $C^1$: $f(t_{j+1},x^{j+1}) = f(t_j+ \Delta t_j, x^j + \Delta t_j f(t_j,x^j)) = f(t_j, x^j) + \Delta t_j \frac{\partial f}{\partial x}(t_j, x^j) + \Delta t_j f(t_j,x^j) \frac{\partial f}{\partial y}(t_j, x^j) + O(\|(\Delta t_j, \Delta t_j f(t_j, x^j))\|^2)$

f est continue donc bornée sur ??? donc : $O(\|(\Delta t_j, \Delta t_j f(t_j, x^j))\|^2) = O({\Delta t_j}^2)$

Ainsi : $[\frac{\partial f}{\partial x}(t_j,x^j)+ f(t_j,x^j)\frac{\partial f}{\partial y}(t_j,x^j)] = \frac{f(t_{j+1},x^{j+1}) - f(t_j, x^j) + O((\Delta t_j)^2)}{\Delta t_j}$
En combinant les deux derniers calculs, on a : $e^{j+1} = [\frac{f(t_{j+1},x^{j+1}) - f(t_j, x^j) + O((\Delta t_j)^2)}{\Delta t_j}] \times \frac{\Delta t_j^2}{2} + O(\Delta t_j^3)$

Comme $e^{j+1} = O((\Delta t_j)^2)$ : pour diminuer l'erreur d'un facteur k il suffit de diminuer le temps d'un facteur $\sqrt{k}$.


On peut adapter le pas de temps en comparant l'erreur locale $e^{j+1}$ et la tolérance d'erreur locale $Tol_{abs}$.

- Si cette erreur est supérieure à la tolérance fixée ($\|e^{j+1}\|$ >  $Tol_{abs}$), il faut diminuer le pas de temps afin d'améliorer l'estimation : comme on a alors $\frac{Tol_{abs}}{\|e^{j+1}\| } < 1$ : on peut actualiser le pas de temps avec la formule $\Delta t_{j+1} = \Delta t_j \sqrt{\frac{Tol_{abs}}{\|e^{j+1}\|}}$.

- Si cette erreur est inférieure à la tolérance fixée ($\|e^{j+1}\|$ <  $Tol_{abs}$), on peut augmenter le pas de temps afin de limiter le temps de simulation : comme on a alors $\frac{Tol_{abs}}{\|e^{j+1}\| } > 1$ : on peut actualiser le pas de temps avec la formule $\Delta t_{j+1} = \Delta t_j \sqrt{\frac{Tol_{abs}}{\|e^{j+1}\|}}$.

La formule $\Delta t_{j+1} = \Delta t_j \sqrt{\frac{Tol_{abs}}{\|e^{j+1}\|}}$ permet ainsi de couvrir toutes les situations.


On nous donne la fonction suivante qui code un schéma d'Euler explicite à pas variable : 

In [ ]:

from numpy import *
import matplotlib; 
from matplotlib.pyplot import *


def solve_ivp_euler_explicit_variable_step(f, t0, x0, t_f, dtmin = 1e-16, dtmax = 0.01, atol = 1e-6):
    dt = dtmax/10; # initial integration step
    ts, xs = [t0], [x0]  # storage variables
    t = t0
    ti = 0  # internal time keeping track of time since latest storage point : must remain below dtmax
    x = x0
    while ts[-1] < t_f:
        while ti < dtmax:
            t_next, ti_next, x_next = t + dt, ti + dt, x + dt * f(x)
            x_back = x_next - dt * f(x_next)
            ratio_abs_error = atol / (linalg.norm(x_back-x)/2)
            dt = 0.9 * dt * sqrt(ratio_abs_error)       ### Formule précédente sur les dt + marge de sécurité par le 0.9
            if dt < dtmin:
                raise ValueError("Time step below minimum")
            elif dt > dtmax/2:      ### on évite que dt augmente trop
                dt = dtmax/2
            t, ti, x = t_next, ti_next, x_next
        dt2DT = dtmax - ti # time left to dtmax
        t_next, ti_next, x_next = t + dt2DT, 0, x + dt2DT * f(x)       ###réinitialisation de ti et retour de t_next à un multiple de dtmax
        ts = vstack([ts,t_next])
        xs = vstack([xs,x_next])
        t, ti, x = t_next, ti_next, x_next
    return (ts, xs)

Cette algorithme permet d'obtenir une solution échantillonée à $\Delta t_{max}$ tout en adaptant le pas de temps dans les calculs afin de limiter l'erreur.

En effet, une première boucle permet de réaliser l'échantillonnage sur la durée $	\lbrack t_0, t_f 	\rbrack $ demandée.
Une seconde boucle sur $t_i$ vient alors limiter le nombre d'échantillons de la solution : on ajoute des valeurs (t, x) que tous les $\Delta t_{max}$ (environ). Ainsi, plusieurs estimations de la solution sont réalisées avec un pas de temps variable afin de limiter l'erreur, mais on conserve une solution échantillonée à $\Delta t_{max}$.

Le calcul du pas de temps est réalisé suivant la formule justifiée précedemment avec un facteur 0.9 supplémentaire afin de conserver une marge de sécurité.


In [ ]:
t1, x1 = solve_ivp_euler_explicit_variable_step(lambda x : x, 0, 1, 5)
t = np.linspace(0, 5)
x = np.exp(t)
plt.figure()
plt.plot(t1, x1, label = 'dtmax = 0.01')
plt.plot(t, x, '--', label = 'exp')
plt.legend()
plt.show()